# Actividad Naive Bayes

En esta actividad, repetiremos el análisis para detección de cancer, usando esta vez Naive-Bayes

## Data


In [ ]:
from sklearn.datasets import load_breast_cancer
loadData = load_breast_cancer(as_frame=True)
data = loadData['data']
data.columns = data.columns.str.replace(' ','_')
target = loadData['target']
target = 1-target # para que "1" sea "Maligno"

Separamos los datos en un set de entrenamiento (`X_train` y `y_train`) para calibrar el modelo, y un set de test (`X_test`, `y_test`) para evaluarlo. (Ojo: no necesitamos normalizar los datos para este método)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=0)


## Parte 1:  Entrenar un modelo Naive-Bayes y evaluar su performance.

Genere un modelo de Naive-Bayes y calibrelo para los datos de entrenamiento.  Con esto haga una predicción para los datos de `X_test`, y guarde estos resultados en la variable `prediccion`

In [ ]:
from sklearn.naive_bayes import GaussianNB
NB = GaussianNB()
NB = NB.fit(<Datos entrenamiento>, <Target entrenamiento>)
prediccion=NB.predict(<Datos test>)

Evalue el performance de su modelo, para las distintas métricas. Para esto, recuerde comparar los resultados reales `y_test` con los resultados predecidos `prediccion`.

Recuerde que puede usar distintas herramientas, por ejemplo:
- Accuracy : `mt.accuracy_score(<reales>,<prediccion>)` 
- Precision: `mt.precision_score(<reales>,<prediccion>)` 
- Recall: `mt.recall_score(<reales>,<prediccion>)` 
- F1: `mt.f1_score(<reales>,<prediccion>)` 
- Reporte general: `mt.classification_report(<reales>,<prediccion>)`
- Matriz confusión: `mt.confusion_matrix(<reales>,<prediccion>)`
- Matriz confusión (plot): `mt.ConfusionMatrixDisplay(mt.confusion_matrix(<reales>,<prediccion>)).plot()`

Recuerde F1= 2TP/(2TP+FN+FP), recall= TP/(TP+FP) y precision= TP/(TP+FN)

In [8]:
from sklearn import metrics as mt
# Escriba aquí sus evaluaciones




## Parte 2 : Repetir la evaluación usando cross-validation
Para major certeza de lo obtenido, haremos un `KFold` (sugerencia: usar 10-fold), y usaremos `cross_validate` para evaluar cada una de las métricas, reportando su media y desviación estandar.  Para esto usamos los datos completos (o sea, no `X_train`, ...) ya que `cross_validate` se encargará de separar el conjunto train y test.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics as mt
from sklearn.model_selection import cross_validate


#Creamos un objeto de la clase KFold 
kf = KFold(n_skf = KFold(n_splits= <NUMERO DE FOLDS ELEGIDO> )

    
#Cree un objeto de la clase GaussianNB. No necesita hacer .fit, cross_validate se encarga de eso
NB = GaussianNB()


#Hacemos cross_validate para las distintas métricas. 
for metrica in ['accuracy', 'recall', 'precision', 'f1']:
    res = cross_validate(NB, <DATOS ORIGINALES>, <TARGET ORIGINAL>, scoring=metrica, cv=kf, return_train_score=True)    
    print ('Score',metrica,'train:', res['train_score'].mean(),'+-',res['train_score'].std())
    print ('Score',metrica,'test:', res['test_score'].mean(),'+-',res['test_score'].std())




## Parte 3: Comparación con los métodos anteriores
Comparemos el performance obtenido para las distintas métricas, con el modelo KNN anteriormente ajustado.  Repita el comando, usando `GaussianNB` esta vez.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics as mt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dataNorm = scaler.fit_transform(data)

#Inicializacion de los resultados
results = pd.DataFrame(columns=['Metodo','Tipo', 'Metrica', 'ScoreMean', 'ScoreStd'])

# KNN
KNNmodel = KNeighborsClassifier(n_neighbors=<EL NUMERO DE VECINOS ELEGIDOS>)
for metrica in ['accuracy', 'recall', 'precision', 'f1']:
    res = cross_validate(KNNmodel, dataNorm, target, cv=kf, return_train_score=True, scoring=metrica)    
    results.loc[len(results)] = ['KNN','train',metrica,res['train_score'].mean(),res['train_score'].std()]
    results.loc[len(results)] = ['KNN','test',metrica,res['test_score'].mean(),res['test_score'].std()]

## Copie lo anterior, y modifiquelo para usar `GaussianNB` esta vez.





Graficamos los resultados

In [ ]:
(ggplot(results)+
 aes(x='Metodo',y="ScoreMean",ymin="ScoreMean-ScoreStd",ymax="ScoreMean+ScoreStd",color='Tipo')+
 geom_point()+ geom_errorbar()+ 
 labs(y="Score")+
 facet_wrap('Metrica')+
 theme_bw()
)






## Parte 4: Interpretación de los resultados.
El modelo Naive-Bayes, ajusta una distribución gaussiana para cada clase (Benigno, Maligno).  Observemos este resultado para entender las decisiones del modelo.

In [ ]:
# Creamos y calibramos nuevamente los datos
NB = GaussianNB()
NB.fit(data, target)

Obtengamos la probabilidad de cada dato para casa clase.

In [ ]:
prediccion_prob = NB.predict_proba(data)

Con esta probabilidad, podemos calcular las medias y desviaciones estandar de cada feature.
- `NB.theta_` retorna las medias de cada clase para cada feature
- `NB.var_` retorna la varianza de cada clase para cada feature

In [ ]:
distrib=pd.DataFrame()
for i in [0,1]:
    for varSeleccionada in range(len(data.columns)):
        simulaciones = np.random.normal(NB.theta_[i][varSeleccionada], np.sqrt(NB.var_[i][varSeleccionada]), 1000)
        tmpDF = pd.DataFrame({'feature':data.columns[varSeleccionada], 'clase': i, 'value':simulaciones})
        distrib=pd.concat([distrib, tmpDF])

y graficamos su distribución.

In [ ]:
(ggplot(distrib, aes(x='value', fill='factor(clase)'))+
 geom_density(alpha=0.5)+
 labs(y="density") + facet_wrap('feature', scales='free')
 + theme(figure_size=(20,20))
)



**Pregunta**: ¿Cuáles son los *features* que parecieran ser mas distintivos para cada clase?